# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846680647452                   -0.70    4.8
  2   -7.852300357538       -2.25       -1.54    1.0
  3   -7.852646017654       -3.46       -2.52    3.0
  4   -7.852646677742       -6.18       -3.39    2.8
  5   -7.852646686284       -8.07       -4.73    1.5
  6   -7.852646686725       -9.36       -5.59    2.8
  7   -7.852646686730      -11.33       -6.58    3.0
  8   -7.852646686730      -13.85       -7.34    1.8
  9   -7.852646686730      -14.57       -8.58    2.2
 10   -7.852646686730   +  -14.57       -9.51    3.5
 11   -7.852646686730      -14.45      -10.29    2.2
 12   -7.852646686730   +    -Inf      -11.41    2.0
 13   -7.852646686730   +  -15.05      -10.22    1.0
 14   -7.852646686730      -14.75      -10.32    1.0
 15   -7.852646686730      -15.05      -10.37    1.0
 16   -7.852646686730   +  -14.45      -10.47    1.0
 17   -7.852646686730      -14.75      -11.26 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846825786115                   -0.70           4.5
  2   -7.852526731298       -2.24       -1.63   0.80    2.0
  3   -7.852636752727       -3.96       -2.71   0.80    1.0
  4   -7.852646481967       -5.01       -3.29   0.80    2.2
  5   -7.852646672312       -6.72       -4.15   0.80    1.0
  6   -7.852646686396       -7.85       -4.81   0.80    1.8
  7   -7.852646686726       -9.48       -5.57   0.80    1.5
  8   -7.852646686729      -11.46       -6.79   0.80    1.5
  9   -7.852646686730      -12.38       -7.21   0.80    2.8
 10   -7.852646686730      -15.05       -8.13   0.80    1.0
 11   -7.852646686730      -14.75       -9.29   0.80    2.0
 12   -7.852646686730   +  -14.57      -10.14   0.80    2.8
 13   -7.852646686730      -14.75      -11.06   0.80    1.8
 14   -7.852646686730      -14.75      -11.18   0.80    2.5
 15   -7.852646686730   +  -14.57      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.392696e+01     3.627924e+00
 * time: 0.6737639904022217
     1     1.371400e+00     1.893717e+00
 * time: 0.9366397857666016
     2    -1.718871e+00     1.836152e+00
 * time: 0.9655358791351318
     3    -3.990718e+00     1.523507e+00
 * time: 1.0066728591918945
     4    -5.480540e+00     1.192447e+00
 * time: 1.0499110221862793
     5    -6.909096e+00     6.999996e-01
 * time: 1.090446949005127
     6    -7.244118e+00     6.618548e-01
 * time: 1.1188969612121582
     7    -7.554681e+00     1.682013e-01
 * time: 1.1479368209838867
     8    -7.668390e+00     1.709347e-01
 * time: 1.1792027950286865
     9    -7.759738e+00     1.434544e-01
 * time: 1.2105128765106201
    10    -7.800248e+00     9.583272e-02
 * time: 1.2398300170898438
    11    -7.828682e+00     6.210079e-02
 * time: 1.2703368663787842
    12    -7.842932e+00     5.058038e-02
 * time: 1.2995028495788574
    13    -7.849778e+00     3.771047e-02
 * time: 1.32998490333

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846812070945                   -0.70    4.5
  2   -7.852310494077       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686666                   -2.54
  2   -7.852646686730      -10.20       -5.84
  3   -7.852646686730   +    -Inf      -11.96
  4   -7.852646686730      -14.75      -15.98


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.463538258820392e-14
|ρ_newton - ρ_scfv| = 1.2087837850343388e-13
|ρ_newton - ρ_dm|   = 7.493056051937786e-10
